# Secure Water Treatment (SWaT)

- Source and description: https://itrust.sutd.edu.sg/itrust-labs_datasets/dataset_info/

In [1]:
from typing import List
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

from config import data_raw_folder, data_processed_folder

from timeeval import Datasets, DatasetManager
from timeeval.datasets import DatasetAnalyzer, DatasetRecord

try:
    from openpyxl import Workbook
except ImportError:
    import sys
    !conda install --yes --prefix {sys.prefix} openpyxl
    from openpyxl import Workbook

In [2]:
plt.rcParams["figure.figsize"] = (20, 10)

In [3]:
source_folder = Path(data_raw_folder) 
target_folder = Path(data_processed_folder)

path_mapping = {
    "SWaT-A1&A2": (
        source_folder / "SWaT.A1&A2_Dec2015" / "Physical" / "SWaT_Dataset_Attack_v0.xlsx",  # test
        source_folder / "SWaT.A1&A2_Dec2015" / "Physical" / "SWaT_Dataset_Normal_v1.xlsx",  # train
    ),
    # "SWaT-A3": "",  # not relevant
    # "SWaT-A4": "SWaT.A4&A5_Jul 2019",  # use A5
    "SWaT-A5": source_folder / "SWaT.A4&A5_Jul2019" / "SWaT_dataset_Jul19v2.xlsx",  # only test = unsupervised
    "SWaT-A6": source_folder / "SWaT.A6_Dec2019" / "csv" / "Dec2019.xlsx",  # only test = unsupervised
    # "SWaT-A7": "",  # not relevant
    # "SWaT-A8": "",  # note relevant
    
}

# define anomalies
anomalies_A5 = {
    "Attack1": {
        "description": "Spoof value of FIT401 from 0.8 to 0.5 to stop de-chlorination by switching off UV401",
        "targets": ["FIT 401"],
        "begin": "2019-07-20 15:08:46+08:00",
        "end": "2019-07-20 15:10:31+08:00"
    },
    "Attack2": {
        "description": " Spoof value of LIT301 from 835 to 1024 to eventually lead to underflow in T301",
        "targets": ["LIT 301"],
        "begin": "2019-07-20 15:15:00+08:00",
        "end": "2019-07-20 15:19:32+08:00"
    },
    "Attack3": {
        "description": "Switch P601 from OFF to ON to increase water in raw water tank",
        "targets": ["P601 Status"],
        "begin": "2019-07-20 15:26:57+08:00",
        "end": "2019-07-20 15:30:48+08:00"
    },
    "Attack4": {
        "description": "Switch from CLOSE to OPEN (MV201) and OFF to ON (P101) to overflow tank T301",
        "targets": ["MV201", "P101 Status"],
        "begin": "2019-07-20 15:38:50+08:00",
        "end": "2019-07-20 15:46:20+08:00"
    },
    "Attack5": {
        "description": "Switch MV501 from OPEN to CLOSE to drain water from RO",
        "targets": ["MV 501"],
        "begin": "2019-07-20 15:54:00+08:00",
        "end": "2019-07-20 15:56:00+08:00"
    },
    "Attack6": {
        "description": "Switch P301 from ON to OFF to halt stage 3 (UF process)",
        "targets": ["P301 Status"],
        "begin": "2019-07-20 16:02:56+08:00",
        "end": "2019-07-20 16:16:18+08:00"
    },
}

for a in anomalies_A5:
    for dt_key in ["begin", "end"]:
        dt = anomalies_A5[a][dt_key]
        # convert times to UTZ-datetimes
        dt = pd.to_datetime(dt).astimezone(tz="UTC")
        
        # fix anomaly labels (timestamps)
        dt = dt - pd.Timedelta(2, unit="m") + pd.Timedelta(14, unit="s")
        anomalies_A5[a][dt_key] = dt

    if a == "Attack1":
        anomalies_A5[a]["begin"] -= pd.Timedelta(2, unit="s")
        anomalies_A5[a]["end"] -= pd.Timedelta(1, unit="s")
    elif a == "Attack2":
        anomalies_A5[a]["begin"] += pd.Timedelta(5, unit="s")
    elif a == "Attack4":
        anomalies_A5[a]["end"] += pd.Timedelta(30, unit="s")
    elif a == "Attack5":
        anomalies_A5[a]["begin"] += pd.Timedelta(3, unit="s")
        anomalies_A5[a]["end"] += pd.Timedelta(40, unit="s")
    elif a == "Attack6":
        anomalies_A5[a]["begin"] -= pd.Timedelta(5, unit="s")
        anomalies_A5[a]["end"] += pd.Timedelta(35, unit="s")

anomalies_A6 = {
    "C2Com&Attack5": {
        "description": "Infiltrate SCADA WS with second payload from C2, and disrupt sensors cycle 1",
        "begin": "12:30",
        "end": "12:33"
    },
    "Attack6": {
        "description": "Disrupt sensors cycle 2",
        "begin": "12:43",
        "end": "12:46"
    },
    "Attack7": {
        "description": "Disrupt sensors cycle 3",
        "begin": "12:56",
        "end": "12:59"
    },
    "Attack8": {
        "description": "Disrupt sensors cycle 4",
        "begin": "13:09",
        "end": "13:12"
    },
    "Attack9": {
        "description": "Disrupt sensors cycle 5",
        "begin": "13:22",
        "end": "13:25"
    }
}

date = "2019-12-06"
tz = "Asia/Hong_Kong"
for a in anomalies_A6:
    for dt_key in ["begin", "end"]:
        # convert times to datetimes
        dt = anomalies_A6[a][dt_key]
        dt = pd.to_datetime(f"{date} {dt}:00").tz_localize(tz=tz)
        anomalies_A6[a][dt_key] = dt

print(f"Looking for source datasets in {Path(source_folder).absolute()} and\nsaving processed datasets in {Path(target_folder).absolute()}")

In [4]:
# shared by all datasets
dataset_collection_name = "SWaT"
dataset_type = "real"
input_type = "multivariate"
datetime_index = True
split_at = None

# create target subfolder
dataset_subfolder = Path(input_type) / dataset_collection_name
target_subfolder = target_folder / dataset_subfolder
target_subfolder.mkdir(parents=True, exist_ok=True)
print(f"Created directories {target_subfolder}")

dm = DatasetManager(target_folder)

In [5]:
def add_anomalies(df: pd.DataFrame, anomalies: dict) -> pd.DataFrame:
    df = df.set_index("timestamp")
    df["is_anomaly"] = 0
    for a in anomalies:
        begin = anomalies[a]["begin"]
        end = anomalies[a]["end"]
        df.loc[begin:end, "is_anomaly"] = 1
    df = df.reset_index()
    return df

def prepare_dataset_A1_A2(dataset_path: Path) -> pd.DataFrame:
    df = pd.read_excel(
        dataset_path,
        sheet_name=0,
        skiprows=1,
        header=0,
        index_col=None,
        parse_dates=[0]
    )
    df = df.rename(columns={" Timestamp": "timestamp"})
    df["is_anomaly"] = 0
    df.loc[df["Normal/Attack"] == "Attack", "is_anomaly"] = 1
    df = df.drop(columns=["Normal/Attack"])
    return df

def prepare_dataset_A5(dataset_path: Path) -> pd.DataFrame:
    df = pd.read_excel(
        dataset_path,
        sheet_name=0,
        skiprows=1,
        header=[0, 1],
        index_col=None,
        parse_dates=[0]
    )
    # align header
    df.columns = list(df.columns.get_level_values(1)[0:1]) + list(df.columns.get_level_values(0)[1:])

    # round timestamps to have equi-distant steps (= seconds)
    df["timestamp"] = df["timestamp"].map(lambda dt: dt.floor(freq="S"))

    # make actuator status numeric
    df = df.replace("Inactive", 0)
    df = df.replace("Active", 1)

    # add anomaly labels
    df = add_anomalies(df, anomalies_A5)
    return df

def prepare_dataset_A6(dataset_path: Path) -> pd.DataFrame:
    df = pd.read_excel(
        dataset_path,
        sheet_name=0,
        skiprows=9,
        header=0,
        index_col=None,
        parse_dates=[0]
    )

    # localize and convert timestamp
    s = df["t_stamp"]
    s = s.map(lambda dt: dt.tz_localize(tz="Asia/Hong_Kong"))
    df.insert(0, "timestamp", s)
    df.drop(columns=["t_stamp"], inplace=True)

    # make actuator status numeric
    df = df.replace("Inactive", 0)
    df = df.replace("Active", 1)

    # add anomaly labels
    df = add_anomalies(df, anomalies_A6)
    return df

## A1 and A2

In [27]:
dataset_name = "SWaT-A1&A2"
train_type = "semi-supervised"
train_is_normal = True

print(f"-> Processing dataset {dataset_name}")

test_filename = f"{dataset_name}.test.csv"
train_filename = f"{dataset_name}.train.csv"
test_path = dataset_subfolder / test_filename
train_path = dataset_subfolder / train_filename
target_test_filepath = target_subfolder / test_filename
target_train_filepath = target_subfolder / train_filename
target_meta_filepath = target_test_filepath.parent / f"{dataset_name}.{Datasets.METADATA_FILENAME_SUFFIX}"

# prepare test dataset
source_paths = path_mapping[dataset_name]
df_test = prepare_dataset_A1_A2(source_paths[0])
df_test.to_csv(target_test_filepath, index=False)
print("  written test dataset")

da = DatasetAnalyzer((dataset_collection_name, dataset_name), is_train=False, df=df_test, ignore_stationarity=True)
da.save_to_json(target_meta_filepath, overwrite=True)
meta = da.metadata
del da
del df_test
print("  analyzed test dataset")

# prepare train dataset
df_train = prepare_dataset_A1_A2(source_paths[1])
df_train.to_csv(target_train_filepath, index=False)
print(f"  written training dataset")

DatasetAnalyzer((dataset_collection_name, dataset_name), is_train=True, df=df_train, ignore_stationarity=True)\
    .save_to_json(target_meta_filepath, overwrite=False)
del df_train
print(f"  analyzed training dataset")

dm.add_dataset(DatasetRecord(
      collection_name=dataset_collection_name,
      dataset_name=dataset_name,
      train_path=train_path,
      test_path=test_path,
      dataset_type=dataset_type,
      datetime_index=datetime_index,
      split_at=split_at,
      train_type=train_type,
      train_is_normal=train_is_normal,
      input_type=input_type,
      length=meta.length,
      dimensions=meta.dimensions,
      contamination=meta.contamination,
      num_anomalies=meta.num_anomalies,
      min_anomaly_length=meta.anomaly_length.min,
      median_anomaly_length=meta.anomaly_length.median,
      max_anomaly_length=meta.anomaly_length.max,
      mean=meta.mean,
      stddev=meta.stddev,
      trend=meta.trend,
      stationarity=meta.get_stationarity_name(),
      period_size=np.nan
))
print(f"... processed dataset {dataset_name}")


## A5

In [6]:
dataset_name = "SWaT-A5"
train_type = "unsupervised"
train_is_normal = False

print(f"-> Processing dataset {dataset_name}")

test_filename = f"{dataset_name}.test.csv"
test_path = dataset_subfolder / test_filename
target_test_filepath = target_subfolder / test_filename
target_meta_filepath = target_test_filepath.parent / f"{dataset_name}.{Datasets.METADATA_FILENAME_SUFFIX}"

# prepare test dataset
source_path = path_mapping[dataset_name]
df_test = prepare_dataset_A5(source_path)
df_test.to_csv(target_test_filepath, index=False)
print("  written test dataset")

da = DatasetAnalyzer((dataset_collection_name, dataset_name), is_train=False, df=df_test)
da.save_to_json(target_meta_filepath, overwrite=True)
meta = da.metadata
del da
del df_test
print("  analyzed test dataset")

dm.add_dataset(DatasetRecord(
      collection_name=dataset_collection_name,
      dataset_name=dataset_name,
      train_path=None,
      test_path=test_path,
      dataset_type=dataset_type,
      datetime_index=datetime_index,
      split_at=split_at,
      train_type=train_type,
      train_is_normal=train_is_normal,
      input_type=input_type,
      length=meta.length,
      dimensions=meta.dimensions,
      contamination=meta.contamination,
      num_anomalies=meta.num_anomalies,
      min_anomaly_length=meta.anomaly_length.min,
      median_anomaly_length=meta.anomaly_length.median,
      max_anomaly_length=meta.anomaly_length.max,
      mean=meta.mean,
      stddev=meta.stddev,
      trend=meta.trend,
      stationarity=meta.get_stationarity_name(),
      period_size=np.nan
))
print(f"... processed dataset {dataset_name}")

## A6

In [ ]:
dataset_name = "SWaT-A6"
train_type = "unsupervised"
train_is_normal = False

print(f"-> Processing dataset {dataset_name}")

test_filename = f"{dataset_name}.test.csv"
test_path = dataset_subfolder / test_filename
target_test_filepath = target_subfolder / test_filename
target_meta_filepath = target_test_filepath.parent / f"{dataset_name}.{Datasets.METADATA_FILENAME_SUFFIX}"

# prepare test dataset
source_path = path_mapping[dataset_name]
df_test = prepare_dataset_A6(source_path)
df_test.to_csv(target_test_filepath, index=False)
print("  written test dataset")

da = DatasetAnalyzer((dataset_collection_name, dataset_name), is_train=False, df=df_test)
da.save_to_json(target_meta_filepath, overwrite=True)
meta = da.metadata
del da
del df_test
print("  analyzed test dataset")

dm.add_dataset(DatasetRecord(
      collection_name=dataset_collection_name,
      dataset_name=dataset_name,
      train_path=None,
      test_path=test_path,
      dataset_type=dataset_type,
      datetime_index=datetime_index,
      split_at=split_at,
      train_type=train_type,
      train_is_normal=train_is_normal,
      input_type=input_type,
      length=meta.length,
      dimensions=meta.dimensions,
      contamination=meta.contamination,
      num_anomalies=meta.num_anomalies,
      min_anomaly_length=meta.anomaly_length.min,
      median_anomaly_length=meta.anomaly_length.median,
      max_anomaly_length=meta.anomaly_length.max,
      mean=meta.mean,
      stddev=meta.stddev,
      trend=meta.trend,
      stationarity=meta.get_stationarity_name(),
      period_size=np.nan
))
print(f"... processed dataset {dataset_name}")

In [28]:
dm.save()

In [29]:
dm.refresh()
dm.df().loc[(slice(dataset_collection_name,dataset_collection_name), slice(None))]

## Exploration

### A1 and A2

In [14]:
df = pd.read_excel(
    source_folder / "SWaT.A1&A2_Dec 2015" / "Physical" / "SWaT_Dataset_Attack_v0.xlsx",
    sheet_name=0,
    skiprows=1,
    header=0,
    index_col=None,
    parse_dates=[0]
)
df["is_anomaly"] = 0
df.loc[df["Normal/Attack"] == "Attack", "is_anomaly"] = 1
df = df.drop(columns=["Normal/Attack"])
df

In [15]:
df_tmp = df[["FIT101", "AIT203", "AIT203", "PIT503", "is_anomaly"]]
df_tmp.iloc[:, :-1].plot()
s = df_tmp["is_anomaly"].diff()
for begin, end in zip(s[s == -1].index, s[s == 1].index):
    plt.gca().add_patch(matplotlib.patches.Rectangle((begin, 0), end - begin, df_tmp.max().max(), color="red", alpha=0.25))
plt.show()

In [16]:
df = pd.read_excel(
    source_folder / "SWaT.A1&A2_Dec 2015" / "Physical" / "SWaT_Dataset_Normal_v1.xlsx",
    sheet_name=0,
    skiprows=1,
    header=0,
    index_col=None,
    parse_dates=[0]
)
df["is_anomaly"] = 0
df.loc[df["Normal/Attack"] == "Attack", "is_anomaly"] = 1
df = df.drop(columns=["Normal/Attack"])
df

In [17]:
df_tmp = df[["FIT101", "LIT101", "AIT203", "FIT201", "LIT401", "PIT503", "is_anomaly"]]
df_tmp.iloc[:, :-1].plot()
s = df_tmp["is_anomaly"].diff()
for begin, end in zip(s[s == -1].index, s[s == 1].index):
    plt.gca().add_patch(matplotlib.patches.Rectangle((begin, 0), end - begin, df_tmp.max().max(), color="red", alpha=0.25))
plt.show()

### A5

In [18]:
df = pd.read_excel(
    source_folder / "SWaT.A4&A5_Jul 2019" / "SWaT_dataset_Jul 19 v2.xlsx",
    sheet_name=0,
    skiprows=1,
    header=[0, 1],
    index_col=None,
    parse_dates=[0]
)

# align header
df.columns = list(df.columns.get_level_values(1)[0:1]) + list(df.columns.get_level_values(0)[1:])

# round timestamps to have equi-distant timesteps (= seconds)
df["timestamp"] = df["timestamp"].map(lambda dt: dt.floor(freq="S"))

df = df.replace("Inactive", 0)
df = df.replace("Active", 1)

df

In [19]:
# define anomalies
anomalies = {
    "Attack1": {
        "description": "Spoof value of FIT401 from 0.8 to 0.5 to stop de-chlorination by switching off UV401",
        "targets": ["FIT 401"],
        "begin": "2019-07-20 15:08:46+08:00",
        "end": "2019-07-20 15:10:31+08:00"
    },
    "Attack2": {
        "description": " Spoof value of LIT301 from 835 to 1024 to eventually lead to underflow in T301",
        "targets": ["LIT 301"],
        "begin": "2019-07-20 15:15:00+08:00",
        "end": "2019-07-20 15:19:32+08:00"
    },
    "Attack3": {
        "description": "Switch P601 from OFF to ON to increase water in raw water tank",
        "targets": ["P601 Status"],
        "begin": "2019-07-20 15:26:57+08:00",
        "end": "2019-07-20 15:30:48+08:00"
    },
    "Attack4": {
        "description": "Switch from CLOSE to OPEN (MV201) and OFF to ON (P101) to overflow tank T301",
        "targets": ["MV201", "P101 Status"],
        "begin": "2019-07-20 15:38:50+08:00",
        "end": "2019-07-20 15:46:20+08:00"
    },
    "Attack5": {
        "description": "Switch MV501 from OPEN to CLOSE to drain water from RO",
        "targets": ["MV 501"],
        "begin": "2019-07-20 15:54:00+08:00",
        "end": "2019-07-20 15:56:00+08:00"
    },
    "Attack6": {
        "description": "Switch P301 from ON to OFF to halt stage 3 (UF process)",
        "targets": ["P301 Status"],
        "begin": "2019-07-20 16:02:56+08:00",
        "end": "2019-07-20 16:16:18+08:00"
    },
}

for a in anomalies:
    for dt_key in ["begin", "end"]:
        # convert times to UTZ-datetimes
        dt = pd.to_datetime(anomalies[a][dt_key]).astimezone(tz="UTC")
        
        # fix anomaly labels (timestamps)
        dt = dt - pd.Timedelta(2, unit="m") + pd.Timedelta(14, unit="s")
        anomalies[a][dt_key] = dt
    if a == "Attack1":
        anomalies[a]["begin"] -= pd.Timedelta(2, unit="s")
        anomalies[a]["end"] -= pd.Timedelta(1, unit="s")
    elif a == "Attack2":
        anomalies[a]["begin"] += pd.Timedelta(5, unit="s")
    elif a == "Attack4":
        anomalies[a]["end"] += pd.Timedelta(30, unit="s")
    elif a == "Attack5":
        anomalies[a]["begin"] += pd.Timedelta(3, unit="s")
        anomalies[a]["end"] += pd.Timedelta(40, unit="s")
    elif a == "Attack6":
        anomalies[a]["begin"] -= pd.Timedelta(5, unit="s")
        anomalies[a]["end"] += pd.Timedelta(35, unit="s")

In [20]:
df = df.set_index("timestamp")

df["is_anomaly"] = 0

for a in anomalies:
    begin = anomalies[a]["begin"]
    end = anomalies[a]["end"]
    df.loc[begin:end, "is_anomaly"] = 1
df = df.reset_index()
df

In [21]:
enlarge_x = 120
for i in range(6):
    a = list(anomalies.keys())[i]
    a = anomalies[a]
    targets = a["targets"]
    begin = a["begin"] - pd.Timedelta(enlarge_x, unit="s")
    end = a["end"] + pd.Timedelta(enlarge_x, unit="s")

    df_tmp = df.set_index("timestamp")
    df_tmp = df_tmp.loc[begin:end, targets + ["is_anomaly"]]
    df_tmp.iloc[:, :-1].plot()
    s = df_tmp["is_anomaly"].diff()
    for begin, end in zip(s[s == -1].index, s[s == 1].index):
        plt.gca().add_patch(matplotlib.patches.Rectangle((begin, 0), end - begin, df_tmp.max().max(), color="red", alpha=0.25))
    plt.show()

### A6

In [22]:
df = pd.read_excel(
    source_folder / "SWaT.A6_Dec 2019" / "csv" / "Dec2019.xlsx",
    sheet_name=0,
    skiprows=9,
    header=0,
    index_col=None,
    parse_dates=[0]
)

# localize timestamp
s = df["t_stamp"]
s = s.map(lambda dt: dt.tz_localize(tz="Asia/Hong_Kong"))
df.insert(0, "timestamp", s)
df.drop(columns=["t_stamp"], inplace=True)

df = df.replace("Inactive", 0)
df = df.replace("Active", 1)

df

In [23]:
anomalies = {
# ignore those attacks, because we could not find any evidence in the sensor data
#    "Infection": {
#        "description": "Infiltrate SCADA WS via USB",
#        "begin": "10:20",
#        "end": "10:30"
#    },
#    "Attack1": {
#        "description": "Exfiltrate data cycle 1",
#        "begin": "10:30",
#        "end": "10:35"
#    },
#    "Attack2": {
#        "description": "Exfiltrate data cycle 2",
#        "begin": "10:45",
#        "end": "10:50"
#    },
#    "Attack3": {
#        "description": "Exfiltrate data cycle 3",
#        "begin": "11:00",
#        "end": "11:05"
#    },
#    "Attack4": {
#        "description": "Exfiltrate data cycle 4",
#        "begin": "11:15",
#        "end": "11:20"
#    },
    "C2Com&Attack5": {
        "description": "Infiltrate SCADA WS with second payload from C2, and disrupt sensors cycle 1",
        "begin": "12:30",
        "end": "12:33"
    },
    "Attack6": {
        "description": "Disrupt sensors cycle 2",
        "begin": "12:43",
        "end": "12:46"
    },
    "Attack7": {
        "description": "Disrupt sensors cycle 3",
        "begin": "12:56",
        "end": "12:59"
    },
    "Attack8": {
        "description": "Disrupt sensors cycle 4",
        "begin": "13:09",
        "end": "13:12"
    },
    "Attack9": {
        "description": "Disrupt sensors cycle 5",
        "begin": "13:22",
        "end": "13:25"
    }
}

date = "2019-12-06"
tz = "Asia/Hong_Kong"

for a in anomalies:
    for dt_key in ["begin", "end"]:
        # convert times to datetimes
        dt = pd.to_datetime(f"{date} {anomalies[a][dt_key]}:00").tz_localize(tz=tz)
        anomalies[a][dt_key] = dt

In [24]:
df = df.set_index("timestamp")

df["is_anomaly"] = 0

for a in anomalies:
    begin = anomalies[a]["begin"]
    end = anomalies[a]["end"]
    df.loc[begin:end, "is_anomaly"] = 1
df = df.reset_index()
df

In [25]:
from sklearn.preprocessing import MinMaxScaler
columns = [c for c in df.columns if c.endswith("Pv")][:5]
df_tmp = df.loc[7000:, columns].copy()
plt.Figure()
df_tmp.plot()
s = df["is_anomaly"].diff()
for begin, end in zip(s[s == -1].index, s[s == 1].index):
    plt.gca().add_patch(matplotlib.patches.Rectangle((begin, 0), end - begin, df_tmp.max().max(), color="red", alpha=0.25))
plt.gca().set_ylim(0, 5)
plt.show()